<h1>Segmenting and Clustering Troronto Neighborhoods</h1>

This Project is part of the IBM Data Science Professional Certificate Caoptone.

<b>Author</b>: Julian Oellrich

<h2 id="libraries">1. Setup and Libraries</h2>

Import Libraries

In [2]:
# Comuptation Libraries
import pandas as pd
import numpy as np

# API Communication
import json
import requests

# Plotting 
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

# Maps and geocoding
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium

# ML Algorithms
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# json & API
import json, lxml
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# data scrping modules
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

# Print Statement
print('Libraries imported!')

Libraries imported!


<h2 id="data_import">2. Data Import</h2>

<h2 id="explore_venues">3. Explore Neighbourhood venues</h2>